In [33]:
import numpy as np
import torch

In [99]:
uncertainty = np.array([[[0.9, 0.2, 0.3], [0.4, 0.5, 0.6], [0.7, 0.8, 0.9]]])
uncertainty

array([[[0.9, 0.2, 0.3],
        [0.4, 0.5, 0.6],
        [0.7, 0.8, 0.9]]])

In [100]:
uncertainty_1 = uncertainty 
uncertainty_1 

array([[[0.9, 0.2, 0.3],
        [0.4, 0.5, 0.6],
        [0.7, 0.8, 0.9]]])

In [101]:
x, y = np.unravel_index(np.argsort((uncertainty - uncertainty.min()), axis=None), uncertainty.shape)
ranking = np.flip(np.stack((x, y), axis=1), 0).tolist()
ranking

ValueError: too many values to unpack (expected 2)

In [102]:
def pointwise_calc_psnr(input_instance, pred_instance, rgb_range=1.0):
    diff = (pred_instance - input_instance) / rgb_range
    mse = diff * diff 
    return -10 * np.log10(mse)

In [104]:
pointwise_calc_psnr(uncertainty, uncertainty_1).mean()

C:\Users\alessandro.lombardin\AppData\Local\Temp\ipykernel_3868\1748816790.py:4: RuntimeWarning: divide by zero encountered in log10
  return -10 * np.log10(mse)


inf

In [74]:
y, x

(array([1, 2, 0, 1, 2, 0, 1, 0, 2], dtype=int64),
 array([0, 0, 1, 1, 1, 2, 2, 0, 2], dtype=int64))

In [13]:
uncertainty[y[0]][x[0]]

0.2